In [1]:
import torch
import numpy as np
import time
import hydroeval as he
from datetime import datetime
from pytz import timezone

In [7]:
def zscore(x):
    x = x.numpy()
    x = np.where((x>0) & (x<1000) == True,x,0)
    np_nanmean = np.nanmean(x)
    np_nanstd = np.nanstd(x)
    zscored = (x-np_nanmean)/np_nanstd
    zscored = np.where(np.isfinite(zscored) == False,0,zscored)
    return zscored

def fill_NOwhiten(x):
    a1 = torch.nanmean(x)
    # b1 = np.nanstd(x)
    y = torch.where(torch.isfinite(x)==False,a1,x)
    # y = (y-a1)/b1
    return y  

class setter2(torch.utils.data.Dataset):
    def __init__(self,x,y,z):
        self.x = x[:z]
        self.y = y[:z,:,:,:6]
        # self.x = torch.load(x)[:z]
        # self.y = torch.load(y)[:z]
        self.std,self.mean = torch.std_mean(self.y)
        self.y = (self.y - self.mean)/self.std
        # self.y = self.y.reshape(self.y.shape[0],self.y.shape[3])
#         rand_pts = torch.from_numpy(np.random.randint(0,self.x.shape[0],1000))
#         self.x = self.x[rand_pts]
#         self.y = self.y[rand_pts]
    def __getitem__(self,idx):
        x = self.x[idx].to('cuda')
        y = self.y[idx].to('cuda')
        return x, y
    def __len__(self):
        return self.x.shape[0]

class a(torch.nn.Module):
    def __init__(self,XXXX,XX):
        super(a,self).__init__()
        chonz = 16
        self.XXXX = XXXX
        self.XX = XX
#         # in layer
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=chonz, kernel_size=3, padding=1, bias=False)
        self.relu = torch.nn.ReLU(inplace=True)
        # hidden layers
        hidden_layers = []
        for i in range(2):
            hidden_layers.append(torch.nn.Conv2d(in_channels=chonz, out_channels=chonz, kernel_size=3, padding=1, bias=False))
            hidden_layers.append(torch.nn.BatchNorm2d(chonz))
            hidden_layers.append(torch.nn.ReLU(inplace=True))
        self.mid_layer = torch.nn.Sequential(*hidden_layers)
        # out layer
        self.conv3 = torch.nn.Conv2d(in_channels=chonz, out_channels=1, kernel_size=3, padding=1, bias=False) #anything below this is for shrinking 
        self.linear1 = torch.nn.Linear(self.XXXX,100) ### this is what gets changed based on important switch
        self.linear2 = torch.nn.Linear(100,50)
        self.linear3 = torch.nn.Linear(50,20)
        self.linear4 = torch.nn.Linear(20,10)
        self.linear5 = torch.nn.Linear(10,self.XX)
    def forward(self, x):
        out1 = self.relu(self.conv1(x))
        out = self.mid_layer(out1)
        o = self.conv3(out+out1)
        o = self.linear1(o.view(o.size(0),-1))
        o = self.relu(self.linear2(o))
        o = self.relu(self.linear3(o))
        o = self.relu(self.linear4(o))
        o = self.linear5(o)
        return o
    
    
class a_simp(torch.nn.Module):
    def __init__(self,XXXX,XX):
        super(a_simp,self).__init__()
        self.XXXX = XXXX
        self.XX = XX
        self.relu = torch.nn.ReLU(inplace=True)
        self.linear1 = torch.nn.Linear(self.XXXX,24) ### this is what gets changed based on important switch
        self.linear2 = torch.nn.Linear(24,12)
        self.linear3 = torch.nn.Linear(50,20)
        self.linear4 = torch.nn.Linear(20,10)
        self.linear5 = torch.nn.Linear(12,self.XX)
    def forward(self, x):
        o = self.linear1(x.view(x.size(0),-1))
        o = self.relu(self.linear2(o))
        # o = self.relu(self.linear3(o))
        # o = self.relu(self.linear4(o))
        o = self.linear5(o)
        return o

In [5]:
imz = torch.load('../data/traintest/COL_MOGL_ZSCORE_traintest.pt')
sf = torch.load('../data/traintest/COL_STFL_traintest.pt')

In [10]:
torch.cuda.empty_cache()
print(f'( {torch.cuda.device_count()} ) {torch.cuda.get_device_name(0)} available')

dset = setter2(imz,sf,210)
# dset = setter2(imz_meanfilled_notwhitened,sf,210)

epochs=200
stop = 1
east = timezone("US/Eastern")
data_split=.8
cube_height = dset.x.shape[2]
cube_width = dset.x.shape[3]
batch_size=2

# train_dset_size = int(data_split*len(dset))
# valid_dset_size = int(len(dset) - train_dset_size)
# train_dset, valid_dset = torch.utils.data.random_split(dset,[train_dset_size,valid_dset_size])


train_size = int(data_split*len(dset))
valid_size = int(0.5*(len(dset) - train_size))
test_size = int(len(dset)-train_size-valid_size)

train_dset = torch.utils.data.TensorDataset(dset.x[:train_size],dset.y[:train_size])
valid_dset = torch.utils.data.TensorDataset(dset.x[train_size: train_size + valid_size],
                                            dset.y[train_size: train_size + valid_size])
test_dset = torch.utils.data.TensorDataset(dset.x[train_size + valid_size:],
                                            dset.y[train_size + valid_size:])
print(len(train_dset),len(valid_dset),len(test_dset))

# train = Subset(dset,range(0,train_dset_size))
# valid = Subset(dset,range(train_dset_size,train_dset_size+valid_dset_size))

train_dataloader = torch.utils.data.DataLoader(dataset=train_dset, batch_size=batch_size, shuffle=False)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dset, batch_size=batch_size, shuffle=False)

model = a(dset.x.shape[2] * dset.x.shape[3],dset.y.shape[-1])
model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
criterion = torch.nn.MSELoss()
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer,step_size=150,verbose=False)

print('Training started at {}'.format(datetime.now(east).strftime('%Y-%m-%d %H:%M:%S')))
t_loss = []
v_loss = []
e_time = []
nse_during = []
t0 = time.time()
for i in range(epochs):
    t_e_loss = 0
    v_e_loss = 0
    train_pred = torch.empty((0,1)).to('cuda')
    train_y = torch.empty((0,1)).to('cuda')
    # val_pred = torch.empty((0,1)).to('cuda')
    # val_y = torch.empty((0,1)).to('cuda')
    t00 = time.time()
    model.train()
    for idx,(x,y) in enumerate(train_dataloader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
 
        predicted = model(x)   
        train_pred = torch.cat((train_pred,predicted.reshape(-1,1)))
        train_y = torch.cat((train_y,y.reshape(-1,1)))
        loss = criterion(predicted.reshape(-1),y.reshape(-1))
        loss.backward()
        optimizer.step()
        t_e_loss += loss.item()
    for xx,yy in valid_dataloader:
        xx = xx.cuda()
        yy = yy.cuda()
        v_pred = model(xx)
        loss2 = criterion(v_pred.reshape(-1),yy.reshape(-1))
        v_e_loss += loss2.item()
        # val_pred = torch.cat((val_pred,v_pred.reshape(-1,1)))
        # val_y = torch.cat((val_y,yy.reshape(-1,1)))
    nse_epoch_train = he.evaluator(he.nse,train_pred.cpu().detach().numpy(),train_y.cpu().detach().numpy())
    # nse_epoch_valid = he.evaluator(he.nse,val_pred.cpu().detach().numpy(),val_y.cpu().detach().numpy())
    if i % 10 == 0:
        print(f"epoch {i} train:\t {np.around(nse_epoch_train,2)}")
    nse_during.append(nse_epoch_train)
    t_loss.append(t_e_loss/len(train_dataloader))
    v_loss.append(v_e_loss/len(valid_dataloader))
    t11 = time.time()
    e_time.append(t11-t00)
    if nse_epoch_train > stop:
        break
t1 = time.time()
print('Training time {} (minutes)'.format(np.format_float_positional((t1-t0)/60,precision=5)))


( 1 ) NVIDIA GeForce GTX 1080 Ti available
168 21 21
Training started at 2023-02-15 15:16:03
epoch 0 train:	 [0.47]
epoch 10 train:	 [0.84]
epoch 20 train:	 [0.93]
epoch 30 train:	 [0.96]
epoch 40 train:	 [0.98]
epoch 50 train:	 [0.98]
epoch 60 train:	 [0.99]
epoch 70 train:	 [0.99]
epoch 80 train:	 [0.99]


KeyboardInterrupt: 